<a href="https://colab.research.google.com/github/jhchang/DFDC/blob/main/Video_prediction_6_collab_with_pruning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Video Face Manipulation Detection Through Ensemble of CNNs
Image and Sound Processing Lab - Politecnico di Milano
- Nicolò Bonettini
- Edoardo Daniele Cannas
- Sara Mandelli
- Luca Bondi
- Paolo Bestagini


# https://github.com/pytorch/tutorials/issues/1054#issuecomment-657991827
# https://adeshpande3.github.io/A-Beginner%27s-Guide-To-Understanding-Convolutional-Neural-Networks/


In [1]:
!git clone https://github.com/polimi-ispl/icpr2020dfdc
!pip install efficientnet-pytorch
!pip install -U git+https://github.com/albu/albumentations > /dev/null
%cd icpr2020dfdc/notebook

Cloning into 'icpr2020dfdc'...
remote: Enumerating objects: 616, done.
remote: Counting objects: 100% (616/616), done.
remote: Compressing objects: 100% (535/535), done.
remote: Total 616 (delta 318), reused 322 (delta 49), pack-reused 0
Receiving objects: 100% (616/616), 68.57 MiB | 21.56 MiB/s, done.
Resolving deltas: 100% (318/318), done.
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-cp37-none-any.whl size=16443 sha256=90b38c0caaca9ca0128a24fad24d26516b207e1831e67872f27dd96b071efdc7
  Stored in directory: /root/.cache/pip/wheels/84/27/aa/c46d23c4e8cc72d41283862b1437e0b3ad318417e8ed7d5921
Successfully built efficientnet-pytorch
  Running command git clone -q https://github.com/albu/albumentations /tmp/pip-req-build-iadw9rbw
/content/icpr2020dfdc/notebook


In [2]:
import torch
from torch.utils.model_zoo import load_url
import matplotlib.pyplot as plt
from scipy.special import expit

import sys
sys.path.append('..')

from blazeface import FaceExtractor, BlazeFace, VideoReader
from architectures import fornet,weights
from isplutils import utils

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pwd

/content/icpr2020dfdc/notebook


In [5]:
import os
from getpass import getpass
import copy
import random
import numpy as np
import time
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F
from itertools import combinations
from pprint import pprint
import json
import csv
import datetime
from sklearn.model_selection import KFold
from functools import lru_cache
import pandas as pd
import io
from contextlib import redirect_stdout
from torchsummary import summary
import pickle

In [6]:
# !pip install mlflow --quiet
# import mlflow

# os.environ['MLFLOW_TRACKING_USERNAME'] = 'jhchang'
# os.environ['MLFLOW_TRACKING_PASSWORD'] = getpass('Enter your DAGsHub access token: ')
# os.environ['MLFLOW_TRACKING_PROJECTNAME'] = 'DFDC_results'

# mlflow.set_tracking_uri(f'https://dagshub.com/' + os.environ['MLFLOW_TRACKING_USERNAME'] + '/' + os.environ['MLFLOW_TRACKING_PROJECTNAME'] + '.mlflow')

In [7]:
video_list = os.listdir('/content/drive/MyDrive/cs274/project_datasets/deepfake-detection-challenge/train_sample_videos/')
video_list = video_list[300:400]
print(video_list)
# video_list = os.listdir('/content/drive/MyDrive/cs274/test_videos/')

['dsndhujjjb.mp4', 'dptrzdvwpg.mp4', 'dtocdfbwca.mp4', 'dtbpmdqvao.mp4', 'drgjzlxzxj.mp4', 'dsdoseflas.mp4', 'dozyddhild.mp4', 'drcyabprvt.mp4', 'dqswpjoepo.mp4', 'dqnyszdong.mp4', 'drsakwyvqv.mp4', 'drtbksnpol.mp4', 'dsjbknkujw.mp4', 'dqppxmoqdl.mp4', 'dqzreruvje.mp4', 'dnyvfblxpm.mp4', 'dzyuwjkjui.mp4', 'ecnihjlfyt.mp4', 'dulanfulol.mp4', 'dvakowbgbt.mp4', 'dxuliowugt.mp4', 'dxuplhwvig.mp4', 'dzieklokdr.mp4', 'dzqwgqewhu.mp4', 'dvumqqhoac.mp4', 'duycddgtrl.mp4', 'ebebgmtlcu.mp4', 'eahlqmfvtj.mp4', 'dzvyfiarrq.mp4', 'duvyaxbzvp.mp4', 'dzwkmcwkwl.mp4', 'dxbqjxrhin.mp4', 'ebkzwjgjhq.mp4', 'ebeknhudxq.mp4', 'eckvhdusax.mp4', 'dubiroskqn.mp4', 'ebywfrmhtd.mp4', 'ebchwmwayp.mp4', 'duzuusuajr.mp4', 'eajlrktemq.mp4', 'dwediigjit.mp4', 'ehccixxzoe.mp4', 'eeyhxisdfh.mp4', 'ecwaxgutkc.mp4', 'eggbjzxnmg.mp4', 'eekozbeafq.mp4', 'ehdkmxgtxh.mp4', 'eebrkicpry.mp4', 'eiriyukqqy.mp4', 'ehieahnhte.mp4', 'edyncaijwx.mp4', 'ehtdtkmmli.mp4', 'ehevsxtecd.mp4', 'eczrseixwq.mp4', 'ehfiekigla.mp4', 'efwfxwwl

In [8]:
try:
    video_list.remove('metadata.json')
except:
    pass

In [9]:
# # video_list
# kf = KFold(n_splits=5)
# test_folds = []

# video_list = np.array(video_list)

# for _, test_index in kf.split(video_list):
#     print("TEST:", test_index)
#     test_folds.append(video_list[test_index])

# pprint(test_folds)

In [10]:
json_truth_dict = {}
with open('/content/drive/MyDrive/cs274/project_datasets/deepfake-detection-challenge/train_sample_videos/metadata.json') as json_file:
# with open('/content/drive/MyDrive/cs274/test_videos/metadata.json') as json_file:
    json_truth_dict = json.load(json_file)
# pprint(json_truth_dict)

## Parameters

In [11]:
"""
Choose an architecture between
- EfficientNetB4
- EfficientNetB4ST
- EfficientNetAutoAttB4
- EfficientNetAutoAttB4ST
- Xception
"""
net_model = 'EfficientNetAutoAttB4'

"""
Choose a training dataset between
- DFDC
- FFPP
"""
train_db = 'DFDC'

In [12]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
face_policy = 'scale'
face_size = 224
frames_per_video = 32

## Initialization

In [13]:
model_url = weights.weight_url['{:s}_{:s}'.format(net_model,train_db)]
net = getattr(fornet,net_model)().eval().to(device)
state_dict = torch.load('/content/drive/MyDrive/cs274/EfficientNetAutoAttB4_DFDC_bestval-72ed969b2a395fffe11a0d5bf0a635e7260ba2588c28683630d97ff7153389fc.pth')
net.load_state_dict(state_dict)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b4-6ed6700e.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b4-6ed6700e.pth



Loaded pretrained weights for efficientnet-b4


<All keys matched successfully>

In [14]:
# print(list(net.modules()))

In [15]:
type(next(net.modules()))

architectures.fornet.EfficientNetAutoAttB4

In [16]:
# save base unpruned model
net_orig = copy.deepcopy(net)

In [17]:
parameters_to_prune = [
    (module, name) 
    for name, module in dict(net.named_modules()).items()
]

params_to_prune_names = []
pruneable_stuff = []
count = 0
for module, name in parameters_to_prune:
    try:
        for layer_type in ['weight', 'bias']:
            prune.global_unstructured([[module, layer_type]], pruning_method=prune.L1Unstructured, amount=0.3)
            params_to_prune_names.append(f"{name}-{layer_type}")
            pruneable_stuff.append([name, layer_type])

    except:
        count += 1
        pass

print("count " + str(count))
print(len(pruneable_stuff))


count 326
420


In [18]:
print(params_to_prune_names)

['efficientnet._conv_stem-weight', 'efficientnet._bn0-weight', 'efficientnet._bn0-bias', 'efficientnet._blocks.0._depthwise_conv-weight', 'efficientnet._blocks.0._bn1-weight', 'efficientnet._blocks.0._bn1-bias', 'efficientnet._blocks.0._se_reduce-weight', 'efficientnet._blocks.0._se_reduce-bias', 'efficientnet._blocks.0._se_expand-weight', 'efficientnet._blocks.0._se_expand-bias', 'efficientnet._blocks.0._project_conv-weight', 'efficientnet._blocks.0._bn2-weight', 'efficientnet._blocks.0._bn2-bias', 'efficientnet._blocks.1._depthwise_conv-weight', 'efficientnet._blocks.1._bn1-weight', 'efficientnet._blocks.1._bn1-bias', 'efficientnet._blocks.1._se_reduce-weight', 'efficientnet._blocks.1._se_reduce-bias', 'efficientnet._blocks.1._se_expand-weight', 'efficientnet._blocks.1._se_expand-bias', 'efficientnet._blocks.1._project_conv-weight', 'efficientnet._blocks.1._bn2-weight', 'efficientnet._blocks.1._bn2-bias', 'efficientnet._blocks.2._expand_conv-weight', 'efficientnet._blocks.2._bn0-weig

In [19]:
# create random permutation of indexes and make a chart of which prune
# yield the best results
# good_params_to_prune_names_comb = list(combinations(good_params_to_prune_names, 2))
# pruneable_stuff_comb = list(combinations(pruneable_stuff, 2))
# pruneable_stuff_comb_randslice = random.sample(range(2, 420), 10)
# print(np.array(good_params_to_prune_names_comb))



In [20]:
prune_percent_pipline = [.80, .70, .60, .50, .40, .30, .20]
param_pipline = [420, 380, 340, 300, 260, 220, 180, 120, 80, 60, 20]
pruneable_stuff_comb_randslice_tail = np.repeat(np.array(param_pipline), len(prune_percent_pipline)).tolist()
pruneable_stuff_comb_randslice = [0] + pruneable_stuff_comb_randslice_tail
prune_percent = [0] + [.80, .70, .60, .50, .40, .30, .20] * len(param_pipline)
# pruneable_stuff_comb_randslice = [0, 1]
# prune_percent = [0, .95]
# pruneable_stuff_comb_randslice = [0, 420, 420]
# prune_percent = [0, .40, .40]
# pruneable_stuff_comb_randslice
print(len(pruneable_stuff_comb_randslice))
print(len(prune_percent))

78
78


In [21]:
random.seed(376)

results = []
column_names = ['pruning_type', 'num_params', 'percentage_pruned', 'accuracy', 'runtime', 'num_videos','num_similar_params', 
                    'total_possible_selected_paramters', 'new_model_size_mb', 'percentage_of_size_saved']
# column_names = ['num_params', 'percentage_pruned', 'accuracy', 'runtime']


pruning_type = 'global_L1unstructured'

now = datetime.datetime.now()
timestamp = str(now.strftime("%m-%d-%Y_%H:%M:%S"))

base_model = True

transf = utils.get_transformer(face_policy, face_size, net.get_normalizer(), train=False)

facedet = BlazeFace().to(device)
facedet.load_weights("../blazeface/blazeface.pth")
facedet.load_anchors("../blazeface/anchors.npy")
videoreader = VideoReader(verbose=False)
video_read_fn = lambda x: videoreader.read_frames(x, num_frames=frames_per_video)
face_extractor = FaceExtractor(video_read_fn=video_read_fn,facedet=facedet)


@lru_cache(maxsize=200)
def get_face(filename):
    vid_faces = face_extractor.process_video('/content/drive/MyDrive/cs274/project_datasets/deepfake-detection-challenge/train_sample_videos/'+ file_name)
    # vid_faces = face_extractor.process_video('/content/drive/MyDrive/cs274/test_videos/'+ file_name)
    return vid_faces

# mlflow.set_experiment("Pruning Model Statistics")

for num_params, prune_p in zip(pruneable_stuff_comb_randslice, prune_percent):
    current_run_module_names_list = []

    pruneable_stuff_comb_pair = random.sample(pruneable_stuff, num_params)

    # print(f'len(pruneable_stuff_comb_pair): {len(pruneable_stuff_comb_pair)}')
    # print(f'len(pruneable_stuff): {len(pruneable_stuff)}')


    # reset the model
    net = copy.deepcopy(net_orig)

    module_list = [
        (module, name) 
        for name, module in dict(net.named_modules()).items()
    ]

    if (not base_model):
        prun_list = []
        current_run_module_names_list = []
        for module, name in module_list:
            for name_target_module in pruneable_stuff_comb_pair:
            # for name_target_module in pruneable_stuff:
                if name == name_target_module[0]:
                    current_run_module_names_list.append(f'{name}_{name_target_module[1]}')
                    prun_list.append([module, name_target_module[1]])
            
        print('# of modules to be pruned: {}'.format(len(current_run_module_names_list)))
        print('% of each module pruned: {}'.format(prune_p))

        # for i in current_run_module_names_list:
        #     param_dict[i] = 0.2

        # print(f'len(prun_list): {len(prun_list)}')
        preprune = []
        pre_count = 0
        for p in prun_list:
            if p[1] == 'weight':
                if p[0].weight != None:
                    temp = np.ravel(p[0].weight.cpu().detach().numpy()).tolist()
                    # print(f"length appended to preprune: {len(temp)}")
                    preprune.extend(temp)
                    pre_count += 1
                else:
                    pass
            elif p[1] == 'bias':
                if p[0].bias != None:
                    temp = np.ravel(p[0].bias.cpu().detach().numpy()).tolist()
                    # print(f"length appended to preprune: {len(temp)}")
                    preprune.extend(temp)
                    pre_count += 1
                else:
                    pass

        # print(f'preprune: {preprune}')
        # print(f'preprune.ndim: {preprune.ndim}')
        preprune = np.ravel(preprune).astype(float)
        # print(f'np.isclose(preprune, 0).sum(): {np.isclose(preprune, 0).sum()}')

        prune.global_unstructured(
            np.array(prun_list),
            pruning_method=prune.L1Unstructured,
            amount=prune_p,
        )

        postprune = []
        post_count = 0
        for p in prun_list:
            if p[1] == 'weight':
                if p[0].weight != None:
                    temp = np.ravel(p[0].weight.cpu().detach().numpy()).tolist()
                    # print(f"length appended to preprune: {len(temp)}")
                    postprune.extend(temp)
                    post_count += 1
                else:
                    pass
            elif p[1] == 'bias':
                if p[0].bias != None:
                    temp = np.ravel(p[0].bias.cpu().detach().numpy()).tolist()
                    # print(f"length appended to preprune: {len(temp)}")
                    postprune.extend(temp)
                    post_count += 1
                else:
                    pass
        postprune = np.ravel(postprune).astype(float)

        # print(f'pre_count: {pre_count}')
        # print(f'post_count: {post_count}')
        # print(f'np.isclose(postprune, 0).sum(): {np.isclose(postprune, 0).sum()}')
        

        num_similar_params = np.isclose(preprune, postprune).sum() 
        print(f'num_similar_params: {num_similar_params}')
        print(f'float(len(preprune)): {float(len(preprune))}')

        total_params_pruned = float(len(preprune)) - num_similar_params

        prune_model_size_diff = total_params_pruned/17550466 * 606.85
        print(f'new model size (MB): {606.85 - prune_model_size_diff} vs. old model size(MB): 606.85 MB')
        prune_model_size_percent_saved = prune_model_size_diff/606.85
        print(f'percentage of size saved: {prune_model_size_percent_saved}')


    else:
        num_similar_params = 17550466
        preprune = []
        prune_model_size_diff = 606.85
        prune_model_size_percent_saved = 0
        print("Running base model")
        base_model = False

    # # Estimate Size

    # f = io.StringIO()
    # with redirect_stdout(f):
    #     summary(net, input_size=(3,224,224))
    # out = f.getvalue()

    # total_params = float(out.split("Total params: ")[-1].split("\n")[0].strip().replace(',',''))
    # params_size = float(out.split("Params size (MB): ")[-1].split("\n")[0].strip().replace(',',''))
    # estimated_total_size = float(out.split("Estimated Total Size (MB): ")[-1].split("\n")[0].strip().replace(',',''))
    
    # fname = str(num_params) + '_' + str(prune_p) + '.pt'
    # torch.save(net, fname)

    # fname = str(num_params) + '_' + str(prune_p) + '.pth'
    # torch.save(net.state_dict(), fname)

    # fname = str(num_params) + '_' + str(prune_p) + '.pkl'
    # with open(fname, 'wb') as file:  
    #     pickle.dump(net, file)
    
    
    # file_size = os.path.getsize(fname)
    # print(f'file size: {file_size} bytes')

    # print(total_params)
    # print(params_size)
    # print(estimated_total_size)

    # for fold in test_folds:

    pred_scores = []
    real_scores = []
    rounded_pred_scores = []

    total_model_run_time = 0
        
        # for file_name in fold:
    # count = 0
    for file_name in video_list:
        # print(f'{count}: {file_name}')
        # count += 1
                    
        vid_faces = get_face(file_name)

        # For each frame, we consider the face with the highest confidence score found by BlazeFace (= frame['faces'][0])
        images = [ transf(image=frame['faces'][0])['image'] for frame in vid_faces if len(frame['faces'])]
        faces_t = torch.stack(images)

        # print("Input Size: {}".format(list(net.parameters())[0].shape))

        run_time_start = time.time()
        with torch.no_grad():
            faces_pred = net(faces_t.to(device)).cpu().numpy().flatten()
        run_time_end = time.time()
        run_time_duration = run_time_end - run_time_start
        total_model_run_time += run_time_duration

        pred_scores.append(expit(faces_pred.mean()))
        rounded_pred_scores.append(round(expit(faces_pred.mean())))

        real_scores.append(json_truth_dict[file_name]['label'] != 'REAL')


    """
    Print average scores.
    An average score close to 0 predicts REAL. An average score close to 1 predicts FAKE.
    """

    real_scores = np.array(real_scores).astype(int)   # actual labels
    rounded_pred_scores = np.array(rounded_pred_scores)   # predicted labels
    correct = (real_scores == rounded_pred_scores)

    accuracy = correct.sum() / correct.size

    # print('Names of params pruned: {}'.format(current_run_module_names_string))

    # print('Files tested: {}'.format(video_list))
    # print('Predicted Scores {}'.format(pred_scores))
    # print('Rounded Predicted Scores{}'.format(rounded_pred_scores))
    # print('Real Scores {}'.format(real_scores))
    print('Accuracy: {}'.format(accuracy))
    print('Runtime: {}'.format(total_model_run_time))

    # print('Total Parameters: {}'.format(total_params))
     # print('Parameters Size: {}'.format(params_size))
    # print('Estimated total size: {}'.format(estimated_total_size))

    current_results = [pruning_type, num_params, prune_p, accuracy, total_model_run_time, len(video_list), num_similar_params, 
                        float(len(preprune)), 606.85 - prune_model_size_diff, prune_model_size_percent_saved] + [1] * len(current_run_module_names_list)

    # print("len(current_results): " + str(len(current_results)))
    # print("len(column_names): " + str(len(column_names)))
    # print(column_names)
    # print(f'len(column_names): {len(column_names)}')
    # print(f'len(current_run_module_names_list): {len(current_run_module_names_list)}')
    # print(f'len(current_results): {len(current_results)}')
    print(list(zip(column_names + current_run_module_names_list,current_results)))

    # with mlflow.start_run() as run:
    #     mlflow.log_params({k:v for k,v in zip(column_names + current_run_module_names_list, current_results)})

    result = pd.Series({k:v for k,v in zip(column_names + current_run_module_names_list, current_results)})

    results.append(result)

    print('='*50)

    df = pd.DataFrame(results)
    df.to_csv(f'/content/drive/MyDrive/cs274/results/{timestamp}.csv')

# df = pd.DataFrame(results, columns=column_names)
# df.to_csv(f'/content/drive/MyDrive/cs274/results/{timestamp}.csv')

    # with mlflow.start_run(run_name="MLflow on Colab"):
    #     mlflow.log_metric("confidence_real", confidence_score_real)
    #     mlflow.log_param("confidence_fake", confidence_score_fake)
    #     mlflow.log_param("run_time", total_model_run_time)
    #     mlflow.log_params(param_dict)

Running base model
Accuracy: 0.9090909090909091
Runtime: 10.306030035018921
[('pruning_type', 'global_L1unstructured'), ('num_params', 0), ('percentage_pruned', 0), ('accuracy', 0.9090909090909091), ('runtime', 10.306030035018921), ('num_videos', 99), ('num_similar_params', 17550466), ('total_possible_selected_paramters', 0.0), ('new_model_size_mb', 0.0), ('percentage_of_size_saved', 0)]
# of modules to be pruned: 420
% of each module pruned: 0.8
num_similar_params: 3510105
float(len(preprune)): 17550466.0
new model size (MB): 121.37040801366754 vs. old model size(MB): 606.85 MB
percentage of size saved: 0.7999993276531803
Accuracy: 0.18181818181818182
Runtime: 10.476350784301758
[('pruning_type', 'global_L1unstructured'), ('num_params', 420), ('percentage_pruned', 0.8), ('accuracy', 0.18181818181818182), ('runtime', 10.476350784301758), ('num_videos', 99), ('num_similar_params', 3510105), ('total_possible_selected_paramters', 17550466.0), ('new_model_size_mb', 121.37040801366754), ('p